In [ ]:
import os
import numpy as np 
import pandas as pd 

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from keras import regularizers
import tensorflow as tf

In [ ]:
cancer = pd.read_csv("Data_GBM/cox_GBM.csv",index_col=0)
data = cancer.transpose()
X_train, X_val = train_test_split(data, test_size=0.2,random_state = seed(7))
ncol = data.shape[1]

In [ ]:
## gbm
en_dim = [250,500,750,1000,1250,1500,1750,2000,2250]
deco = [1250,1500,1750,2000,2250]

In [ ]:
## Undercomplete AE 
for x in en_dim:
    for y in deco:
        encoding_dim = x
        input_dim = Input(shape = (ncol, ))
        encoded1 = Dense(y, activation = 'tanh')(input_dim)
        encoded2 = Dense(encoding_dim, activation = 'tanh')(encoded1)
        decoded1 = Dense(y, activation = 'tanh')(encoded2)
        decoded2 = Dense(ncol, activation = 'sigmoid')(decoded1)
        autoencoder = Model(inputs = input_dim, outputs = decoded2)
        autoencoder.compile(optimizer = 'adam', loss = 'mean_squared_error')
        autoencoder.fit(X_train, X_train, epochs = 100, batch_size = 256, shuffle = False, validation_data = (X_val, X_val))
        encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoded_train = pd.DataFrame(encoder.predict(data))
        encoded_train.to_csv("Data_GBM/Results/undercomplete_gbm_"+str(x)+"_"+str(y)+".csv",index=False)      

In [ ]:
regu = [0.1,0.01,0.001,0.0001,0.00001,0.000001]
X_train, X_test = train_test_split(data, train_size = 0.8, random_state = seed(1993))

In [ ]:
## Sparse AE
for x in en_dim:
    for y in deco:
        for z in regu:
            encoding_dim = x
            encoded2 = Dense(y, activation = 'tanh',activity_regularizer=regularizers.l1(l1=z))(input_dim)
            encoded3 = Dense(encoding_dim, activation = 'tanh')(encoded2)
            decoded1 = Dense(y, activation = 'tanh',activity_regularizer=regularizers.l1(l1=z))(encoded3)
            decoded2 = Dense(ncol, activation = 'sigmoid')(decoded1)
            autoencoder = Model(inputs = input_dim, outputs = decoded2)
            autoencoder.compile(optimizer = 'adam', loss = 'binary_crossentropy')
            autoencoder.fit(X_train, X_train, epochs = 100, batch_size = 256, shuffle = False, validation_data = (X_test, X_test))
            encoder = Model(inputs = input_dim, outputs = encoded3)
            encoded_input = Input(shape = (encoding_dim, ))
            encoded_train = pd.DataFrame(encoder.predict(data))
            encoded_train.to_csv("Data_GBM/Results/sparse_gbm_"+str(x)+"_"+str(y)+"_"+str(z)+".csv", index=False)